In [1]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import glob
import os
from os import *
import re
import imutils
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from scipy import ndimage
import collections
import tqdm

In [2]:
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]


In [3]:
#p = '/home/jovyan/DATA_MASTER_PROJECT/FINAL_Data/3_classes_1_t_MEDIUM/train_full_size/'

In [ ]:
t = []
for i in listdir(p):
    im =cv2.imread(p+i)
    if np.count_nonzero(im == 255) < 7000:
        t.append(im)

In [ ]:
len(t)

In [4]:
def get_time_point(path, time_point_1,time_point_2 ,time_point_3,time_point_4,time_point_5,time_point_6):   # time_point in form of 'Take #1 ' (remember the space between #1 ') or 'Take #32'

    image_time_point = []
    well_name = []

    for dirname, dirnames,filenames in os.walk(path):



        for filename in sorted(filenames, key = natural_keys):



            if time_point_1 in filename or time_point_2 in filename or time_point_3 in filename or time_point_4 in filename or time_point_5 in filename or time_point_5 in filename:



                im = cv2.imread(os.path.join(dirname, filename))
                
                if np.count_nonzero(im == 255) < 30000000:
        
                
            
                    image_time_point.append(im)
                    well_name.append(filename)

    return image_time_point, well_name   

In [5]:
def find_and_cut(path_img,time_point_1,time_point_2,time_point_3,time_point_4,time_point_5,time_point_6, save_path): #'Take #1 ', # "{0}/sk_{1}_exp_2_time_1.tiff"

    t_1,s = get_time_point(path_img,time_point_1,time_point_2,time_point_3,time_point_4,time_point_5,time_point_6)

    tot_dict = {}

    for idx, image in tqdm.tqdm(enumerate(t_1)):

        dict_1 = {}

        cx = []
        cy = []

        shifted = cv2.pyrMeanShiftFiltering(image, 10, 50)
        gray = cv2.cvtColor(shifted,cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
        D = ndimage.distance_transform_edt(thresh)
        localMax = peak_local_max(D, indices=False, min_distance=9, labels=thresh)
        markers = ndimage.label(localMax, structure=np.ones((3, 3)))[0]
        labels = watershed(-D, markers, mask=thresh)



        for label in np.unique(labels):
            if label == 0:
                continue




            mask = np.zeros(gray.shape, dtype='uint8')
            mask[labels == label] = 255
            cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            cnts = imutils.grab_contours(cnts)
            c = max(cnts, key=cv2.contourArea)
            ((x, y), r) = cv2.minEnclosingCircle(c)



            cx.append(int(x))
            cy.append(int(y))


            data = {'X':cx,'Y':cy}

            data = pd.DataFrame(data)


        dict_1[idx] = data

        tot_dict.update(dict_1)


    t_1_1,s = get_time_point(path_img, time_point_1,time_point_2,time_point_3, time_point_4,time_point_5,time_point_6)


    value_dict = []

    for k, v in sorted(tot_dict.items()):
        value_dict.append(v)


    name_well = []
    
    for i in s:
        name_well.append(i.split(' ')[1].split(' ')[0])
    
    
    
    crop_cell_1 = []
    
    n_well = []

    for idx , (i,n) in enumerate(zip(t_1_1,name_well)):

        img = t_1_1[idx]
    
        st = name_well[idx]

        x = []
        y = []

        for i in range(len(value_dict[idx])):
            x.append(value_dict[idx].iloc[i,0])      
            y.append(value_dict[idx].iloc[i,1])


        w = 45
        h = 45

        for ix, i in enumerate(x):

            crop = img[(y[ix]-h):(y[ix]+h), (x[ix]-w):(x[ix]+w)]

            if crop.size == 24300:
                
                
                crop_cell_1.append(crop)
                n_well.append([st])



    na_well = [s for i in n_well for s in i]
    for ls, (i,_) in enumerate(zip(crop_cell_1,na_well)):

        im = Image.fromarray(i)

        im.save(save_path.format(via,na_well[ls], ls))   # exp 1 or 2 sonce they did dublicate studies 

    return len(crop_cell_1), len(na_well)

In [6]:
via = '/home/jovyan/DATA_MASTER_PROJECT/FINAL_Data/3_classes_t_96_HIGH/croppped_test'
directory = os.mkdir(via) 

In [7]:
p = '/home/jovyan/DATA_MASTER_PROJECT/FINAL_Data/3_classes_t_96_HIGH/test_full_size/'
path_img = p


In [12]:
croped_cell, name = find_and_cut(path_img, 'E10','B11','D8','B2','NONE','NONE' , "{0}/HRH_{1}_{2}.tiff")

croped_cell, name

144it [01:35,  1.51it/s]


(6463, 6463)

In [14]:


a0 = [['controll', 0]] * 8391

a1 =[['ADR', 1]] * (10761 + 8762)

a2 =[['HRH', 2]] * (11081 + 6463)

#a3 =[['mast', 3]] * 50

# a4 =[['misoprostol', 4]] * 8371

# a5 =[['molindone', 5]] * 6766

# a6 =[['nadol', 6]] * 7420

# a7 =[['nadol', 7]] * 7420



labels = a1+a0+a2#+a2#+a2+a3+a4+a5+a6



In [15]:
df = pd.DataFrame(labels, columns = ['Type', 'Category'])
df = df.sort_values(by=['Type'])

In [16]:

df.to_csv("/home/jovyan/DATA_MASTER_PROJECT/FINAL_Data/3_classes_t_96_HIGH/test_labels.csv", sep=',',index=False)

In [ ]:
labels